In [1]:
import TwitterFactCheck
import TwitterAPI
import importlib
import pandas as pd
import re
import os
import numpy as np
import json
import spacy
import logging
from collections import Counter
import networkx as nx
from networkx.algorithms import community
import altair as alt
from vega_datasets import data
import matplotlib.pyplot as plt
importlib.reload(TwitterAPI)
importlib.reload(TwitterFactCheck)
logging.basicConfig(level=logging.INFO)

# 1. Load the data and filtering

In [2]:
def find_loc(location):
    for city in geo["Cities"]:
        if city.lower() in location.lower():
            return geo["Cities"][city]
    for county in geo["Counties"]:
        if county.lower() in location.lower():
            return geo["Counties"][county]
    for key, val in geo["States"].items():
        if key in location or val.lower() in location.lower():
            return val
    return np.NaN

In [4]:
df_users = pd.read_csv("Data/Network/NetworkUsers.csv", sep="\t", index_col="User_id")

In [13]:
df_users = pd.read_csv("Data/Network/NetworkUsers.csv", sep="\t", index_col="User_id")
with open("Data/GeoInfo.json", "r") as f:
    geo = json.loads(f.read())
df_users["state"] = df_users["Location"].astype(str).apply(find_loc)
df_loc = df_users.dropna(subset="state").groupby(["state"]).count().reset_index()
df_loc["id"] = df_loc["state"].apply(geo["FIPS"].get).astype(int)

In [14]:
states = alt.topo_feature(data.us_10m.url, 'states')
alt.Chart(states).mark_geoshape().transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_loc, 'id', ["Username", "id"])
).encode(
    color="Username:Q",
).project(type='albersUsa') 

alt.Chart(...)

# 3. Build the network and detect the communities

In [27]:
%%time
authors = set()
edges = list()
df_conn = pd.read_csv("Data/Network/NetworkTweets.csv", sep="\t", index_col="id").dropna(subset={"u1", "u2"})
df_conn["author1"] = df_conn["u1"].apply(lambda x: df_users.loc[x]["Username"])
df_conn["author2"] = df_conn["u2"].apply(lambda x: df_users.loc[x]["Username"])
df_conn = df_conn.dropna(subset={"author1", "author2"})
# for i, row in df_conn.iterrows():
#     a1, a2 = row.author1, row.author2
#     authors = authors | {a1, a2}
#     edges.append([a1, a2])
# author_dict = {author: i for i, author in enumerate(sorted(authors))}
# edges = [(author_dict[a1], author_dict[a2]) for a1, a2 in edges]
# edges = dict(Counter(edges))

<timed exec>:3: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.


KeyError: 7.660577483164221e+17

In [ ]:
#  Build the community in unweighted graph
G = nx.Graph()
G.add_edges_from(edges.keys())
comm_uw = community.greedy_modularity_communities(G)
plt.figure(figsize=(12,12))
nx.draw(G, with_labels=True, node_size=500)
# nx.write_gexf(G, "Graph/unweighted.gexf")

In [ ]:
#  Build the community in weighted graph
G = nx.Graph()
for edge, weight in edges.items():
    G.add_edge(*edge, weight=weight)
comm = community.greedy_modularity_communities(G, "weights")
plt.figure(figsize=(15,15))
nx.draw(G, with_labels=True, node_size=500)
# nx.write_gexf(G, "Graph/weighted.gexf")

# 4. Find someone popular

In [45]:
ct = Counter(df_conn.u1)
ct.update(df_conn.u2)
df_hot = pd.DataFrame(pd.Series(ct).sort_values(ascending=False), columns=["count"])
df_hot["username"] = df_users["username"]

In [48]:
df_hot.iloc[:10]

,count,username
20545835,42430,newsmax
2836421,38556,MSNBC
8953122,11376,PolitiFact
21925564,9022,PogueMoran
980526530,7666,hvnacuba76
1691491086,7552,Craig1454
520817923,6769,DimensioT
70394965,6749,citizentvkenya
632859279,6604,RadioCitizenFM
312149882,6028,Chris_1791
